In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import math

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle
def calculate_2d_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)
    
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def process_video(file_path, class_name, df):
    cap = cv2.VideoCapture(file_path)

    pose = mp_pose.Pose()
    frame_data = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            try:
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                       landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]



                right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                
                
                left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)

                
            except IndexError:
                right_shoulder_angle = np.nan
                right_elbow_angle = np.nan
                right_hip_angle = np.nan
                right_knee_angle = np.nan
                left_shoulder_angle = np.nan
                left_elbow_angle = np.nan
                left_hip_angle = np.nan
                left_knee_angle = np.nan
        else:
            right_shoulder_angle = np.nan
            right_elbow_angle = np.nan
            right_hip_angle = np.nan
            right_knee_angle = np.nan
            left_shoulder_angle = np.nan
            left_elbow_angle = np.nan
            left_hip_angle = np.nan
            left_knee_angle = np.nan

        frame_data.append({
            'frame': cap.get(cv2.CAP_PROP_POS_FRAMES),
            'right_shoulder_angle' : right_shoulder_angle,
            'right_elbow_angle' : right_elbow_angle,
            'right_hip_angle' : right_hip_angle,
            'right_knee_angle' : right_knee_angle,
            'left_shoulder_angle' : left_shoulder_angle,
            'left_elbow_angle' : left_elbow_angle,
            'left_hip_angle' : left_hip_angle,
            'left_knee_angle' : left_knee_angle,
            'class_name': class_name
        })

    cap.release()
    df = pd.concat([df, pd.DataFrame(frame_data)], ignore_index=True)
    return df

if __name__ == "__main__":
    video_folder = 'vid'  
    output_folder = 'csv'  

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    
    all_data = pd.DataFrame()

    
    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0]  
            file_path = os.path.join(video_folder, file_name)

            
            all_data = process_video(file_path, class_name, all_data)

    output_file = os.path.join(output_folder, 'output.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Saved all angles to csv\output.csv


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical


data = pd.read_csv('csv/output.csv')


#data.dropna(inplace=True)


label_encoder = LabelEncoder()
data['class_name'] = label_encoder.fit_transform(data['class_name'])


features = data[['right_shoulder_angle',
            'right_elbow_angle',
            'right_hip_angle',
            'right_knee_angle',
            'left_shoulder_angle',
            'left_elbow_angle',
            'left_hip_angle',
            'left_knee_angle']].values

labels = data['class_name'].values


scaler = StandardScaler()
features = scaler.fit_transform(features)

sequence_length = 40


def create_sequences(data, labels, seq_length):
    sequences = []
    labels_seq = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        label_seq = labels[i + seq_length - 1]
        sequences.append(seq)
        labels_seq.append(label_seq)
    return np.array(sequences), np.array(labels_seq)

X, y = create_sequences(features, labels, sequence_length)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=12, validation_data=(X_test, y_test))


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

model.save('lstm_model2.h5')
print('Model saved as lstm_model.h5')


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.8892 - loss: 0.3755 - val_accuracy: 0.9968 - val_loss: 0.0193
Epoch 2/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9964 - loss: 0.0166 - val_accuracy: 0.9819 - val_loss: 0.0412
Epoch 3/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9976 - loss: 0.0092 - val_accuracy: 0.9960 - val_loss: 0.0195
Epoch 4/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9908 - loss: 0.0348 - val_accuracy: 0.9984 - val_loss: 0.0058
Epoch 5/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.9990 - loss: 0.0048 - val_accuracy: 0.9944 - val_loss: 0.0145
Epoch 6/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9902 - loss: 0.0440 - val_accuracy: 0.9992 - val_loss: 0.0029
Epoch 7/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.9990 - loss: 0.0046 - val_accuracy: 0.9992 - val_loss: 0.0030
Epoch 8/30
828/828 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9996 - loss: 0.0021 - val_accuracy

Test Accuracy: 99.96%
Model saved as lstm_model.h5


In [4]:
from sklearn.preprocessing import LabelEncoder
import joblib

joblib.dump(label_encoder, 'label_encoder.pkl')


['label_encoder.pkl']